In [1]:
import pandas as pd
import string
import unicodedata
import sys
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import sklearn.model_selection as cv
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
plt.style.use("fivethirtyeight")
%matplotlib inline

In [2]:
smileyfaces = [':-)', ':)', ':D', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)']
sadfaces = ['>:[', ':-(', ':(', ':-c', ':c', ':-<', ':<', ':-[', ':[', ':{', '=(','=[', 'D:']
angryfaces = ['>:(', '(╯°□°)╯︵ ┻━┻']
cryingfaces = [":’-(", ":’("]
skepticalfaces = ['>:', '>:/', ':-/', '=/',':L', '=L', ':S', '>.<']
noexpressionfaces = [':|', ':-|', '(｀・ω・´)']
surprisedfaces = ['>:O', ':-O', ':O', ':-o', ':o', '8O', 'O_O', 'o-o', 'O_o', 'o_O', 'o_o', 'O-O']

In [3]:
def cleanTextU(wordSeries):
    tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                          if unicodedata.category(chr(i)).startswith('P'))
    def remove_punctuation(text):
        return text.translate(tbl)
    for smile in smileyfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(smile, ' smileyface '))
    for sad in sadfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(sad,' sadface '))
    for angry in angryfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(angry, ' angryface '))
    for cry in cryingfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(cry, ' cryingface '))
    for skeptical in skepticalfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(skeptical, ' skepticalface '))
    for noexp in noexpressionfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(noexp, ' noexpressionfaces '))
    for surprised in surprisedfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(surprised, ' surprisedface '))
    wordSeries = wordSeries.apply(lambda x: x.replace('...', ' dotdotdot '))
    wordSeries = wordSeries.apply(lambda x: x.replace('!', ' exclamatory '))
    wordSeries = wordSeries.apply(lambda x: remove_punctuation(x))
    wordSeries = wordSeries.apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
    wordSeries = wordSeries.apply(lambda x: x.lower())
    wordSeries = wordSeries.apply(lambda x: x.replace('<br >',' '))
    wordSeries = wordSeries.apply(lambda x: x.replace('<br>',' '))
    wordSeries = wordSeries.apply(lambda x: x.replace('`',''))
    wordSeries = wordSeries.apply(lambda x: x.replace(' id ', ' '))
    wordSeries = wordSeries.apply(lambda x: x.replace(' im ', ' '))
    #wordSeries = wordSeries.apply(lambda x: x.replace('http', ' '))
    wordSeries = wordSeries.apply(lambda x: ' '.join( [w for w in x.split() if len(w)>1] ))
    return wordSeries

In [4]:
def cleanText(wordSeries):
    def remove_punctuation(x):
        for char in string.punctuation:
            x = x.replace(char, ' ')
        return x
    for smile in smileyfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(smile, ' smileyface '))
    for sad in sadfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(sad,' sadface '))
    for angry in angryfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(angry, ' angryface '))
    for cry in cryingfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(cry, ' cryingface '))
    for skeptical in skepticalfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(skeptical, ' skepticalface '))
    for noexp in noexpressionfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(noexp, ' noexpressionfaces '))
    for surprised in surprisedfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(surprised, ' surprisedface '))
    wordSeries = wordSeries.apply(lambda x: x.replace('...', ' dotdotdot '))
    wordSeries = wordSeries.apply(lambda x: x.replace('!', ' exclamatory '))
    wordSeries = wordSeries.apply(lambda x: remove_punctuation(x))
    wordSeries = wordSeries.apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
    wordSeries = wordSeries.apply(lambda x: x.lower())
    #wordSeries = wordSeries.apply(lambda x: x.replace('http', ' '))
    wordSeries = wordSeries.apply(lambda x: ' '.join( [w for w in x.split() if len(w)>1] ))
    return wordSeries

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
def tokenize(documents, unicode):
    if unicode:
        documents = cleanTextU(documents)
    else:
        documents = cleanText(documents)
    docs = [word_tokenize(content) for content in documents]
    stopwords_=set(stopwords.words('english'))
    def filter_tokens(sent):
        return([w for w in sent if not w in stopwords_])
    docs=list(map(filter_tokens,docs))
    lemmatizer = WordNetLemmatizer()
    docs_lemma = [[lemmatizer.lemmatize(word) for word in words] for words in docs]
    return docs_lemma

In [6]:
dataPN = pd.read_csv('text_polarity.csv', encoding = "ISO-8859-1", header=None)
dataS = pd.read_csv('text_emotion.csv')

In [7]:
dataPN.drop(dataPN.index[400000:1200000], inplace = True)

In [9]:
dataPN[0] = dataPN[0].apply(lambda x: 1 if x == 4 else 0)

In [7]:
dataPN['Tokens'] = tokenize(dataPN[5], True)

In [8]:
dataPN['Tokens'] = dataPN['Tokens'].apply(lambda x: ' '.join(x))
corpus = [row for row in dataPN['Tokens']]
tfidf = TfidfVectorizer()
document_tfidf_matrix = tfidf.fit_transform(corpus)

In [9]:
X = document_tfidf_matrix
y = dataPN[0]

In [1]:
lg = LogisticRegression(max_iter = 800)
lg.fit(X,y)

NameError: name 'LogisticRegression' is not defined

In [15]:
dataS['Tokens'] = tokenize(dataS['content'], False)
dataS['tokenJ'] = dataS['Tokens'].apply(lambda x: ' '.join(x))
corpusS = [row for row in dataS['tokenJ']]
document_tfidf_matrixS = tfidf.transform(corpusS)

In [16]:
document_tfidf_matrixS

<40000x436686 sparse matrix of type '<class 'numpy.float64'>'
	with 306344 stored elements in Compressed Sparse Row format>

In [17]:
Xs = document_tfidf_matrixS
ys = dataS['sentiment']
#X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size = 0.5)

In [34]:
lg.predict(Xs)

array([0, 0, 0, ..., 0, 1, 1])

In [35]:
dataS['polarity'] = lg.predict(Xs)

In [36]:
positiveS = ['enthusiasm', 'neutral', 'surprise', 'love', 'fun', 'happiness', 'relief']
negativeS = ['empty', 'sadness', 'neutral', 'worry', 'hate', 'boredom', 'anger']

In [37]:
dataP = dataS[dataS['sentiment'].isin(positiveS)]
dataN = dataS[dataS['sentiment'].isin(negativeS)]

In [38]:
dataP['Tokens'] = tokenize(dataP['content'], False)
dataN['Tokens'] = tokenize(dataN['content'], False)

/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
dataP['tokenJ'] = dataP['Tokens'].apply(lambda x: ' '.join(x))
corpusP = [row for row in dataP['tokenJ']]
tfidfP = TfidfVectorizer()
Xp = tfidfP.fit_transform(corpusP)

/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
dataN['tokenJ'] = dataN['Tokens'].apply(lambda x: ' '.join(x))
corpusN = [row for row in dataN['tokenJ']]
tfidfN = TfidfVectorizer()
Xn = tfidfN.fit_transform(corpusN)

/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
yn = dataN['sentiment']
yp = dataP['sentiment']

In [42]:
lgP = LogisticRegression(max_iter = 300)
lgN = LogisticRegression(max_iter = 300)
lgP.fit(Xp, yp)
lgN.fit(Xn, yn)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
testP = dataS[dataS['polarity'] == 1]
testN = dataS[dataS['polarity'] == 0]

In [46]:
testP['Tokens'] = tokenize(testP['content'], False)
testN['Tokens'] = tokenize(testN['content'], False)

/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
testP['tokenJ'] = testP['Tokens'].apply(lambda x: ' '.join(x))
corpusP = [row for row in testP['tokenJ']]
Xtestp = tfidfP.transform(corpusP)

/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
testN['tokenJ'] = testN['Tokens'].apply(lambda x: ' '.join(x))
corpusN = [row for row in testN['tokenJ']]
Xtestn = tfidfN.transform(corpusN)

/Users/brooks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
ytestp = testP['sentiment']
ytestn = testN['sentiment']

In [51]:
lgP.score(Xtestp, ytestp)

0.47528850367629155

In [52]:
lgN.score(Xtestn, ytestn)

0.528284437137132

In [74]:
def getTopN(n, reg, X, y, moods):
    probs = reg.predict_proba(X)
    topN = []
    for prob in probs:
        best_N = list(reversed(np.argsort(prob)))[:n]
        topN.append(best_N)
    topN = np.array(topN)
    mood = np.array(sorted(moods))
    topNpred = mood[topN]
    return topNpred

In [75]:
def getScore(X, y):
    count = 0
    for idx in range(len(y)):
        if np.array(y)[idx] in X[idx]:
            count+=1
    return count/len(y)

In [82]:
XpredsP = getTopN(3, lgP, Xtestp, ytestp, positiveS)
getScore(XpredsP, ytestp)

0.7205044553732288

In [83]:
XpredsN = getTopN(3, lgN, Xtestn, ytestn, negativeS)
getScore(XpredsN, ytestn)

0.7363201972974361